In [1]:
import json
import hashlib
from datetime import datetime
import urllib
import pandas as pd
import numpy as np
import math
import datetime as dt
import yfinance as yf

In [2]:
import logging

# Create a logger object
log = logging.getLogger()
log.setLevel(logging.DEBUG)  # Set the root logger level to DEBUG


# Create a console handler for printing logs to console
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)  # Set console handler level to INFO

# Define format for both handlers
formatter = logging.Formatter('[%(asctime)s][%(levelname)s] - %(message)s')
console_handler.setFormatter(formatter)

# Add the handlers to the logger

log.addHandler(console_handler)



In [38]:
import yfinance as yf

# Ticker symbol for ITC
ticker_symbol = "^NSEBANK"  # For the National Stock Exchange of India

# Fetch historical data
data = yf.download(ticker_symbol, period="4d", interval="1m")
data['Change']=data['Close']-data['Open']
data['Pre_Change']= (data['Change']/data['Open'])*100


In [39]:
old_price_long = 0
old_price_short = 0
target_prencentage = 0.15
stop_loss_precentage = 1
stop_loss_long = False
stop_loss_short = False
stop_loss_price_long = 0
stop_loss_price_short = 0
long_hold = False
short_hold = False
trade_book = []
qty = 30
brokerage = 20*1.18
other_charges_long = (0.004/100)*1.18
other_charges_short = 0.0149769/100
stop_long = False
stop_short = False




for index, row in data.iterrows():

    log.debug(f"variables values when running on_ticks getting ticks :- old_price_long : {old_price_long},old_price_short : {old_price_short},target_prencantage : {target_prencentage},stop_loss_precentage : {stop_loss_precentage},long_hold : {long_hold},short_hold : {short_hold},trade_book : {trade_book},qty : {qty}")
    #records_ticks(ticks['datetime'],ticks['open'],ticks['high'],ticks['low'],ticks['close'])

    

    current_price = row['Close']
    date = index
    total_charges_long = current_price*qty*other_charges_long+brokerage
    total_charges_short = current_price*qty*other_charges_short+brokerage

    per_change_long = ((current_price-old_price_long)/old_price_long)*100 if old_price_long != 0 else 0
    per_change_short = ((current_price-old_price_short)/old_price_short)*100 if old_price_short != 0 else 0
    price_change_long = current_price-old_price_long
    price_change_short = current_price-old_price_short
    log.info(f"% Change in Price for Long position = {per_change_long}")
    log.info(f"% Change in Price for Short position = {per_change_short} NOTE :: Negetive means profit in short trade same formula has been used to both long and short") 
    
    if (per_change_long >= target_prencentage) and long_hold :

        log.info(f"Exiting Long Trade since Both the Condition Satisfied per_change_long = {per_change_long} is greater than target_precentage = {target_prencentage} and Holding Position for Long is = {long_hold}")
        long_hold = False
        log.debug(f"setting long_hold to False.value after setting false = {long_hold}")
        profit_long = price_change_long
        log.info(f"Sold at {current_price} * {qty} = {current_price*qty}. Booked Profit of {profit_long} * {qty} = {profit_long*qty}\n")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':profit_long*qty,'charges':total_charges_short,'net_profit':profit_long*qty-total_charges_short})
        log.debug(f"Appended to Trade book ('date':{date},'type':'exit long','price':{current_price},'order_value':{int(order_value)},'profit':{profit_long*qty})")
        stop_long = True
        if stop_long and stop_short:
            break

    if per_change_long <= -stop_loss_precentage and long_hold:

        log.warning(f"Stoploss on Long Position Triggered since Both Condition per_change_long = {per_change_long} is greater than Stoploss = {stop_loss_precentage} in absolute values and Long_hold is True = {long_hold}. NOTE per_change_long should be negetive since it is decreasing.")
        long_hold = False
        log.debug(f"Assigned False value to long_hold : {long_hold}")
        stop_loss_long = True
        log.debug(f"Changed stop_loss_long value to True since it is triggered. new value = {stop_loss_long}")
        loss = current_price-old_price_long
        log.warning(f"Exiting Long Position Booked a LOSS of {loss} * {qty} = {loss*qty}\n")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'exit long STOPLOSS','price':current_price,'order_value':int(order_value),'profit':loss*qty,'charges':total_charges_short,'net_profit':loss*qty-total_charges_short})
        log.debug(f"Appended to Trade book ('date':{date},'type':'exit long STOPLOSS','price':{current_price},'order_value':{int(order_value)},'profit':{loss*qty})")

    if (not long_hold) and (not stop_loss_long) and (not stop_long):

        log.info(f"Going Long Since not Holding and neither stoploss is triggered values :: long_hold : {long_hold}, stop_loss_long : {stop_loss_long}")
        long_hold = True

        log.info(f'Buying at a price {current_price} * {qty}. total buy price {qty*current_price} \n ')
        old_price_long = current_price
        log.debug(f"Setting global old_price_long to {current_price} and long_hold to {long_hold}")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'long','price':current_price,'order_value':int(order_value),'profit':0,'charges':total_charges_long,'net_profit':0-total_charges_long})
        


    
    if (per_change_short <= -target_prencentage) and short_hold:
        log.debug(f"Exiting Short Trade since Both Condition Satisfied. per_change_short : {per_change_short} is greater than target_prencentage : {target_prencentage} and short hold is True Value : {short_hold}")
        short_hold = False
        log.debug(f'Set short_hold to False new Value = {short_hold}')
        profit_short = old_price_short-current_price 
        log.info(f"Exiting Trade at price {current_price} * qty {qty} = {current_price*qty}")
        log.info(f"Booked a profit of {profit_short} * {qty} = {profit_short*qty}")
        order_value = qty*current_price
        
        trade_book.append({'date':date,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':profit_short*qty,'charges':total_charges_long,'net_profit':profit_short*qty-total_charges_long})
        stop_short = True
        
        if stop_long and stop_short:
            break
        



    if per_change_short >= stop_loss_precentage and short_hold:
        log.warning(f"Stoploss on Short Position Triggered since per_change_short : {per_change_short} is greater than stop_loss_precentage : {stop_loss_precentage}. in absolute term. and short_hold is True : {short_hold} NOTE :: per_change_should be positive since it is a short trade")
        short_hold = False
        log.debug('Assigned new value to short_hold False : {False}')
        stop_loss_short = True
        log.debug(f'Since Stoploss triggered setting stop_loss_short to Truew new value : {stop_loss_short}')
        loss_on_short = old_price_short-current_price
        log.warning(f'Exiting short Trade at price : {current_price}. Booked a loss of {loss_on_short} *qty = {loss_on_short*qty}')
        
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'exit short Stoploss','price':current_price,'order_value':int(order_value),'profit':loss_on_short*qty,'charges':total_charges_long,'net_profit':loss_on_short*qty-total_charges_long})

    
    if (not short_hold) and (not stop_loss_short) and (not stop_short):
        
        log.info(f"Going Short since Both short_hold : {short_hold} and stop_loss_short : {stop_loss_short} are False. ")
        short_hold = True
        log.info(f'Shorting at a price {current_price}. total buy price {qty*current_price} \n ')
        old_price_short = current_price
        log.debug(f"Setting old_short_price to {current_price}. value after setting {old_price_short}")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'short','price':current_price,'order_value':int(order_value),'profit':0,'charges':total_charges_short,'net_profit':0-total_charges_short})

    if index.hour==15 and index.minute==29:
        if long_hold:
            long_hold = False
            profit_long = current_price-old_price_long
            order_value = qty*current_price
            trade_book.append({'date':index,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':profit_long*qty,'charges':total_charges_short,'net_profit':(profit_long*qty)-total_charges_short})


        if short_hold:
            short_hold = False
            profit_short = old_price_short-current_price
            order_value = qty*current_price
            trade_book.append({'date':index,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':profit_short*qty,'charges':total_charges_long,'net_profit':(profit_short*qty)-total_charges_long})
            

        break

[2024-03-24 00:35:20,413][INFO] - % Change in Price for Long position = 0
[2024-03-24 00:35:20,414][INFO] - % Change in Price for Short position = 0 NOTE :: Negetive means profit in short trade same formula has been used to both long and short
[2024-03-24 00:35:20,415][INFO] - Going Long Since not Holding and neither stoploss is triggered values :: long_hold : False, stop_loss_long : False
[2024-03-24 00:35:20,415][INFO] - Buying at a price 46524.8515625 * 30. total buy price 1395745.546875 
 
[2024-03-24 00:35:20,416][INFO] - Going Short since Both short_hold : False and stop_loss_short : False are False. 
[2024-03-24 00:35:20,417][INFO] - Shorting at a price 46524.8515625. total buy price 1395745.546875 
 
[2024-03-24 00:35:20,419][INFO] - % Change in Price for Long position = 0.006548919335953012
[2024-03-24 00:35:20,419][INFO] - % Change in Price for Short position = 0.006548919335953012 NOTE :: Negetive means profit in short trade same formula has been used to both long and short


In [40]:
trade_book_df = pd.DataFrame(trade_book)
trade_book_df

,date,type,price,order_value,profit,charges,net_profit
0,2024-03-19 09:15:00+05:30,long,46524.851562,1395745,0.000000,89.479190,-89.479190
1,2024-03-19 09:15:00+05:30,short,46524.851562,1395745,0.000000,232.639415,-232.639415
2,2024-03-19 09:59:00+05:30,exit short,46426.000000,1392780,2965.546875,89.339216,2876.207659
3,2024-03-19 15:29:00+05:30,exit long,46333.101562,1389993,-5752.500000,231.777869,-5984.277869


In [41]:
print(sum(trade_book_df['profit']))
print(sum(trade_book_df['charges']))
print(sum(trade_book_df['net_profit']))

-2786.953125
643.2356892598438
-3430.188814259844


In [82]:
for index,row in data.iterrows():
    print(index,row['Close'])

2024-03-15 09:15:00+05:30 46667.3515625
2024-03-15 09:16:00+05:30 46716.75
2024-03-15 09:17:00+05:30 46750.69921875
2024-03-15 09:18:00+05:30 46754.25
2024-03-15 09:19:00+05:30 46757.8984375
2024-03-15 09:20:00+05:30 46749.55078125
2024-03-15 09:21:00+05:30 46758.80078125
2024-03-15 09:22:00+05:30 46792.3515625
2024-03-15 09:23:00+05:30 46769.19921875
2024-03-15 09:24:00+05:30 46780.1484375
2024-03-15 09:25:00+05:30 46762.3984375
2024-03-15 09:26:00+05:30 46749.94921875
2024-03-15 09:27:00+05:30 46744.30078125
2024-03-15 09:28:00+05:30 46717.8515625
2024-03-15 09:29:00+05:30 46729.19921875
2024-03-15 09:30:00+05:30 46736.8515625
2024-03-15 09:31:00+05:30 46744.5
2024-03-15 09:32:00+05:30 46708.6015625
2024-03-15 09:33:00+05:30 46703.3984375
2024-03-15 09:34:00+05:30 46676.25
2024-03-15 09:35:00+05:30 46645.6015625
2024-03-15 09:36:00+05:30 46633.75
2024-03-15 09:37:00+05:30 46585.6015625
2024-03-15 09:38:00+05:30 46589.6015625
2024-03-15 09:39:00+05:30 46550.5
2024-03-15 09:40:00+05:30

In [ ]:
class scalping():

    def __init__(self,qty,data):
        self.data = data
        self.old_price_long = 0
        self.old_price_short = 0
        self.target_prencentage = 0.01
        self.stop_loss_precentage = 1
        self.stop_loss_long = False
        self.stop_loss_short = False
        self.long_hold = False
        self.short_hold = False
        self.trade_book = []
        self.qty = qty
        self.brokerage = 20*1.18
        self.other_charges_long = (0.004/100)*1.18
        self.other_charges_short = 0.0149769/100

    def long_target():


        if (per_change_long >= target_prencentage) and long_hold :
            log.info(f"Exiting Long Trade since Both the Condition Satisfied per_change_long = {per_change_long} is greater than target_precentage = {target_prencentage} and Holding Position for Long is = {long_hold}")
            long_hold = False
            log.debug(f"setting long_hold to False.value after setting false = {long_hold}")
            profit_long = price_change_long
            log.info(f"Sold at {current_price} * {qty} = {current_price*qty}. Booked Profit of {profit_long} * {qty} = {profit_long*qty}\n")
            order_value = qty*current_price
            trade_book.append({'date':date,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':profit_long*qty,'charges':total_charges_short,'net_profit':profit_long*qty-total_charges_short})
            log.debug(f"Appended to Trade book ('date':{date},'type':'exit long','price':{current_price},'order_value':{int(order_value)},'profit':{profit_long*qty})")
            stop_long = True
            if stop_long and stop_short:
                break
    
    def long_stoploss():
        if per_change_long <= -stop_loss_precentage and long_hold:

            log.warning(f"Stoploss on Long Position Triggered since Both Condition per_change_long = {per_change_long} is greater than Stoploss = {stop_loss_precentage} in absolute values and Long_hold is True = {long_hold}. NOTE per_change_long should be negetive since it is decreasing.")
            long_hold = False
            log.debug(f"Assigned False value to long_hold : {long_hold}")
            stop_loss_long = True
            log.debug(f"Changed stop_loss_long value to True since it is triggered. new value = {stop_loss_long}")
            loss = current_price-old_price_long
            log.warning(f"Exiting Long Position Booked a LOSS of {loss} * {qty} = {loss*qty}\n")
            order_value = qty*current_price
            trade_book.append({'date':date,'type':'exit long STOPLOSS','price':current_price,'order_value':int(order_value),'profit':loss*qty,'charges':total_charges_short,'net_profit':loss*qty-total_charges_short})
            log.debug(f"Appended to Trade book ('date':{date},'type':'exit long STOPLOSS','price':{current_price},'order_value':{int(order_value)},'profit':{loss*qty})")

    def long_initiate():
        if (not long_hold) and (not stop_loss_long) and (not stop_long):

            log.info(f"Going Long Since not Holding and neither stoploss is triggered values :: long_hold : {long_hold}, stop_loss_long : {stop_loss_long}")
            long_hold = True

            log.info(f'Buying at a price {current_price} * {qty}. total buy price {qty*current_price} \n ')
            old_price_long = current_price
            log.debug(f"Setting global old_price_long to {current_price} and long_hold to {long_hold}")
            order_value = qty*current_price
            trade_book.append({'date':date,'type':'long','price':current_price,'order_value':int(order_value),'profit':0,'charges':total_charges_long,'net_profit':0-total_charges_long})
            


    def short_target():
        if (per_change_short <= -target_prencentage) and short_hold:
            log.debug(f"Exiting Short Trade since Both Condition Satisfied. per_change_short : {per_change_short} is greater than target_prencentage : {target_prencentage} and short hold is True Value : {short_hold}")
            short_hold = False
            log.debug(f'Set short_hold to False new Value = {short_hold}')
            profit_short = old_price_short-current_price 
            log.info(f"Exiting Trade at price {current_price} * qty {qty} = {current_price*qty}")
            log.info(f"Booked a profit of {profit_short} * {qty} = {profit_short*qty}")
            order_value = qty*current_price
            
            trade_book.append({'date':date,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':profit_short*qty,'charges':total_charges_long,'net_profit':profit_short*qty-total_charges_long})
            stop_short = True
            
            if stop_long and stop_short:
                break
        


    def short_stoploss():
        if per_change_short >= stop_loss_precentage and short_hold:
            log.warning(f"Stoploss on Short Position Triggered since per_change_short : {per_change_short} is greater than stop_loss_precentage : {stop_loss_precentage}. in absolute term. and short_hold is True : {short_hold} NOTE :: per_change_should be positive since it is a short trade")
            short_hold = False
            log.debug('Assigned new value to short_hold False : {False}')
            stop_loss_short = True
            log.debug(f'Since Stoploss triggered setting stop_loss_short to Truew new value : {stop_loss_short}')
            loss_on_short = old_price_short-current_price
            log.warning(f'Exiting short Trade at price : {current_price}. Booked a loss of {loss_on_short} *qty = {loss_on_short*qty}')
            
            order_value = qty*current_price
            trade_book.append({'date':date,'type':'exit short Stoploss','price':current_price,'order_value':int(order_value),'profit':loss_on_short*qty,'charges':total_charges_long,'net_profit':loss_on_short*qty-total_charges_long})

    
    def short_initiate():
        if (not short_hold) and (not stop_loss_short) and (not stop_short):
            
            log.info(f"Going Short since Both short_hold : {short_hold} and stop_loss_short : {stop_loss_short} are False. ")
            short_hold = True
            log.info(f'Shorting at a price {current_price}. total buy price {qty*current_price} \n ')
            old_price_short = current_price
            log.debug(f"Setting old_short_price to {current_price}. value after setting {old_price_short}")
            order_value = qty*current_price
            trade_book.append({'date':date,'type':'short','price':current_price,'order_value':int(order_value),'profit':0,'charges':total_charges_short,'net_profit':0-total_charges_short})


    def stop_trade():
        if index.hour==15 and index.minute==29:
            if long_hold:
                long_hold = False
                profit_long = current_price-old_price_long
                order_value = qty*current_price
                trade_book.append({'date':index,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':profit_long*qty,'charges':total_charges_short,'net_profit':(profit_long*qty)-total_charges_short})


            if short_hold:
                short_hold = False
                profit_short = old_price_short-current_price
                order_value = qty*current_price
                trade_book.append({'date':index,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':profit_short*qty,'charges':total_charges_long,'net_profit':(profit_short*qty)-total_charges_long})
                


In [10]:
import pandas as pd

# Assuming 'data' is your DataFrame containing stock market data with datetime as the index

# Group data by date
grouped = data.groupby(data.index.date)

import yfinance as yf

# Replace 'AAPL' with the stock symbol you're interested in
stock_symbol = 'AAPL'

# Fetch one year of historical data
data = yf.download(stock_symbol, start='2023-03-21', end='2024-03-21')

print(data)



for date, data_group in grouped:
    open_price = data_group.iloc[0]['Close']  # Get the open price for the day
    max_price = open_price * 1.001  # Calculate 0.1% increase from open price
    min_price = open_price * 0.99  # Calculate 1% decrease from open price

    for index, row in data_group.iterrows():
        if row['Close'] <= min_price:
            print(f"On {date}, price decreased more than 1% before hitting 0.1% increase from open price.")
            break  # Exit inner loop if condition met
        elif row['Close'] >= max_price:
            print(f"On {date}, price hit 0.1% increase from open price.")
            break  # Exit inner loop if condition met


On 2024-03-14, price hit 0.1% increase from open price.
On 2024-03-15, price hit 0.1% increase from open price.
On 2024-03-18, price hit 0.1% increase from open price.
On 2024-03-19, price hit 0.1% increase from open price.
On 2024-03-20, price hit 0.1% increase from open price.
On 2024-03-21, price hit 0.1% increase from open price.
On 2024-03-22, price hit 0.1% increase from open price.


In [24]:
import pandas as pd

# Assuming 'data' is your DataFrame containing stock market data with datetime as the index
import yfinance as yf

# Replace 'AAPL' with the stock symbol you're interested in
stock_symbol = '^NSEBANK'

# Fetch one year of historical data
data = yf.download(stock_symbol, interval='1m',period='7d')

data = pd.DataFrame(data)
print(data)







timestamps_list = []

# Group data by date
grouped = data.groupby(data.index.date)

for date, data_group in grouped:
    open_price = data_group.iloc[0]['Close']  # Get the open price for the day
    max_price = open_price * 1.001  # Calculate 0.1% increase from open price
    min_price = open_price * 0.99  # Calculate 1% decrease from open price

    increase_hit = False
    decrease_hit = False

    for index, row in data_group.iterrows():
        if not increase_hit and row['Close'] >= max_price:
            timestamps_list.append({'date': str(date), 'time': str(index.time()), 'event': 'increase'})
            increase_hit = True
        elif not decrease_hit and row['Close'] <= min_price:
            timestamps_list.append({'date': str(date), 'time': str(index.time()), 'event': 'decrease'})
            decrease_hit = True

        if increase_hit and decrease_hit:
            break  # Exit inner loop if both conditions are met

# Convert the list of dictionaries to a DataFrame
timestamps = pd.DataFrame(timestamps_list)

print("Timestamps of either increase or decrease:")
print(timestamps)


                                   Open          High           Low  \
Datetime                                                              
2024-03-14 09:15:00+05:30  46825.750000  46834.449219  46711.898438   
2024-03-14 09:16:00+05:30  46731.898438  46731.898438  46625.750000   
2024-03-14 09:17:00+05:30  46671.000000  46679.648438  46593.500000   
2024-03-14 09:18:00+05:30  46606.800781  46633.898438  46577.800781   
2024-03-14 09:19:00+05:30  46582.300781  46628.398438  46573.351562   
...                                 ...           ...           ...   
2024-03-22 15:25:00+05:30  46851.398438  46860.101562  46841.550781   
2024-03-22 15:26:00+05:30  46849.601562  46855.351562  46832.898438   
2024-03-22 15:27:00+05:30  46836.351562  46866.050781  46834.250000   
2024-03-22 15:28:00+05:30  46855.351562  46855.351562  46831.898438   
2024-03-22 15:29:00+05:30  46841.550781  46861.148438  46830.550781   

                                  Close     Adj Close  Volume  
Datetime    